## Cleans TRACE Corporate Bond Data

In [1]:
# Imports
import pandas as pd
import numpy as np

In [2]:
# Loads in subset of prof Yoshio's data
with pd.read_csv("prof_yoshio_data/trace_credit_spreads.csv", chunksize=1000) as reader:
    data = reader.get_chunk()

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   cusip_id                1000 non-null   object 
 1   date                    1000 non-null   int64  
 2   price                   1000 non-null   float64
 3   volume                  1000 non-null   int64  
 4   ISSUE_ID                1000 non-null   int64  
 5   ISSUER_ID               1000 non-null   int64  
 6   PROSPECTUS_ISSUER_NAME  1000 non-null   object 
 7   MATURITY                1000 non-null   int64  
 8   SECURITY_LEVEL          1000 non-null   object 
 9   OFFERING_AMT            1000 non-null   int64  
 10  OFFERING_DATE           1000 non-null   int64  
 11  OFFERING_PRICE          852 non-null    float64
 12  INTEREST_FREQUENCY      1000 non-null   int64  
 13  COUPON                  1000 non-null   float64
 14  standard                1000 non-null   i

In [174]:
# data.memory_usage(deep=True) \
# .gt(8000)

As expected all "object" (ie. string) columns use up the majority of the memory. The rest (float/int64s) are using 8 bytes, however these may also be reduced to 16 or 32.

In [13]:
def cols(col_names, data_type):
    '''
    Takes in list of column names and a datatype and generates a dictionary
    '''
    return {col_names[i]: data_type for i in range(len(col_names))}

In [19]:
def clean_data_types(df):
    '''
    Cleans datatypes for raw data
    '''
    df.columns = map(str.lower, df.columns)
    
    # String/text columns
    string_cols = ['cusip_id', 'cusip', 'prospectus_issuer_name']
    strings = cols(string_cols, 'string[pyarrow]')

    bool_cols = ['callable', 'make_whole', 'conv', 'junior']
    bools = cols(bool_cols, 'bool')

    category_cols = ['security_level', 'rating_spr', 'rating_mdy', 'standard', 'putop', 'industry_code']
    categories = cols(category_cols, 'category')
    try:
        df = df.drop(["sp_rat", "moody_rat", "rating", 'rating_date_spr', 'rating_date_mdy'], axis=1)
    except: pass
        
    # Float/Integer columns
    int8_cols = ['interest_frequency']
    int8s = cols(int8_cols, 'int8')

    df['sic_code'] = df['sic_code'].fillna(0)
    int16_cols = ['sic_code']
    int16s = cols(int16_cols, 'int16')

    int32_cols = ['volume', 'offering_amt', 'amount_outstanding', 'issue_id', 'issuer_id']
    int32s = cols(int32_cols, 'int32')

    float32_cols = ['price', 'offering_price', 'coupon', 'accrued_interest', 'tau',
                   'age', 'ytm', 'duration', 'tr_dirty_price', 'tr_ytm', 'cs']
    float32s = cols(float32_cols, 'float32')

    # Dates
    # formula to convert to datetime: df['date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
    df['offering_date'] = df['offering_date'].fillna(0)
    date_cols = ['date', 'maturity', 'offering_date', 'month1']
    dates = cols(date_cols, 'int32')

    # Applies type changes
    types = strings | bools | categories | int8s | int16s | int32s | float32s | dates

    return df.astype(types)

In [192]:
df = clean_data_types(df)
# df.memory_usage(deep=True)

In [198]:
df.info()
# Memory usage decreased by about half

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   cusip_id                1000 non-null   string  
 1   date                    1000 non-null   int32   
 2   price                   1000 non-null   float32 
 3   volume                  1000 non-null   int32   
 4   issue_id                1000 non-null   int32   
 5   issuer_id               1000 non-null   int32   
 6   prospectus_issuer_name  1000 non-null   string  
 7   maturity                1000 non-null   int32   
 8   security_level          1000 non-null   category
 9   offering_amt            1000 non-null   int32   
 10  offering_date           1000 non-null   int32   
 11  offering_price          852 non-null    float32 
 12  interest_frequency      1000 non-null   int8    
 13  coupon                  1000 non-null   float32 
 14  standard                1

In [204]:
# Loads in whole data
data = pd.read_csv("prof_yoshio_data/trace_credit_spreads.csv")

C:\Users\sroma\AppData\Local\Temp\ipykernel_25248\3165405631.py:2: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("prof_yoshio_data/trace_credit_spreads.csv")


14090424.827 KB


In [221]:
# Cleans data
cleaned_data = clean_data_types(data)

14090424.827 KB


In [ ]:
# Saves cleaned data
cleaned_data.to_csv('cleaned_data.csv')

In [222]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18833993 entries, 0 to 18833992
Data columns (total 40 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   cusip_id                object 
 1   date                    int64  
 2   price                   float64
 3   volume                  float64
 4   issue_id                int64  
 5   issuer_id               int64  
 6   prospectus_issuer_name  object 
 7   maturity                int64  
 8   security_level          object 
 9   offering_amt            float64
 10  offering_date           float64
 11  offering_price          float64
 12  interest_frequency      int64  
 13  coupon                  float64
 14  standard                int64  
 15  putop                   int64  
 16  conv                    int64  
 17  junior                  int64  
 18  cusip                   object 
 19  callable                object 
 20  make_whole              object 
 21  industry_code           int64

In [224]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18833993 entries, 0 to 18833992
Data columns (total 35 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   cusip_id                string  
 1   date                    int32   
 2   price                   float32 
 3   volume                  int32   
 4   issue_id                int32   
 5   issuer_id               int32   
 6   prospectus_issuer_name  string  
 7   maturity                int32   
 8   security_level          category
 9   offering_amt            int32   
 10  offering_date           int32   
 11  offering_price          float32 
 12  interest_frequency      int8    
 13  coupon                  float32 
 14  standard                category
 15  putop                   category
 16  conv                    bool    
 17  junior                  bool    
 18  cusip                   string  
 19  callable                bool    
 20  make_whole              bool    
 21  indust

# Aggregates Data to Monthly

In [2]:
# Imports
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *

In [132]:
# Loads in subset of cleaned data
with pd.read_csv("cleaned_data.csv", chunksize=100_000, index_col=0) as reader:
    data = reader.get_chunk()

# csv doesn't preserve data types
data = clean_data_types(data)

In [133]:
print(f''' With the subset of data there are: 
    - {len(data['issuer_id'].unique())} unique issuers 
    - {len(data['issue_id'].unique())} unique bonds  
    - {len(data['date'].sort_values().unique())} days in the sample
    ''')

 With the subset of data there are: 
    - 43 unique issuers 
    - 192 unique bonds  
    - 4913 days in the sample
    


## Sample Exclusion Criteria
Based on Bai, Bali & Wen (2019):

1. Remove bonds that are not listed or traded in the US public market, which include bonds issued through private placement, bonds issued under the 144A rule, bonds that do not trade in US dollars, and bond issuers not in the jurisdiction of the United States.
2. Remove bonds that are structured notes, mortgage backed or asset backed, agency backed, or equity linked.
3. Remove convertible bonds since this option feature distorts the return calculation and makes it impossible to compare the returns of convertible and nonconvertible bonds.
4. Remove bonds that trade under 5 or above 1000. 
5. Remove bonds that have a floating coupon rate, which means the sample comprises only bonds with a fixed or zero coupon. This rule is applied based on the consideration of the accuracy in bond return calculation, given the challenge in tracking a floating-coupon bond’s cash flows.
6. Remove bonds that have less than one year to maturity. This rule is applied to all major corporate bond indices such as the Barclays Capital Corporate Bond Index, the Bank of America Merrill Lynch Corporate Master Index, and the Citi Fixed Income Indices. If a bond has less than one year to maturity, it will be delisted from major bond indices; hence, index-tracking investors will change their holding positions. This operation will distort the return calculation for bonds with less than one year to maturity; thus, we remove them from our sample.
7. For intraday data, we also eliminate bond transactions that are labeled as when-issued, locked-in, or have special sales conditions, and that have more than a twoday settlement.
8. Remove transaction records that are canceled and adjust records that are subsequently corrected or reversed.
9. Remove transaction records that have trading volume less than $10,000

### 3. Convertible Bonds

In [156]:
def filter_bond_convertible(data):
    '''
    Function which filters out convertible bonds.
    '''
    
    return data[data['conv'] == False]

In [157]:
data = filter_bond_convertible(data)

### 4. Bonds with irregular trading prices

In [162]:
def filter_bond_prices(data):
    '''
    Function which filters out bonds that trade below $5 or above $1000.
    '''
    
    return data[(data['price'].gt(5)) | (data['price'].lt(1000))]

In [163]:
data = filter_bond_prices(data)

### 6. Bonds with less than 1-year to maturity
Perhaps worthwhile to keep seperate as a robustness check?

In [134]:
def filter_bond_maturity(data):
    '''
    Function which filters out bonds with less than 1-year to maturity.
    Saves bonds with less than 1-year to maturity in a seperate file.
    '''
    data['date'] = pd.to_datetime(data['date'], format='%Y%m%d')
    data['maturity'] = pd.to_datetime(data['maturity'], format='%Y%m%d')

    data = data.sort_values(by='date')
    data['days_to_maturity'] = data['maturity'] - data['date']

    # Filters out bonds with less than 1-year to maturity
    data_one_year_bonds = data[data['days_to_maturity'].le('365 days')]
    data_one_year_bonds.to_csv('bonds_1_year_maturity.csv')
    
    return data[data['days_to_maturity'].gt('365 days')]

In [135]:
data = filter_bond_maturity(data)

In [137]:
data.shape

(96721, 36)

### 9. Bonds with small trading volumes

In [143]:
def filter_bond_volume(data):
    '''
    Function which filters daily volumes of less than $10,000.
    '''
    
    return data[data['volume'].gt(10000)]

In [144]:
data = filter_bond_volume(data)

In [146]:
data.shape

(95529, 36)